# Extract minimal historic_data db dump
If you don´t want to use full db (because of size), you can build some minimal dump file with this notebook.
Which historic_data entries you like to keep, you can control via the regex used. 
By default `2023-11-29 1.*t.*2023-11-29` extracts all entries recorded at _2023-11-29_ in an hour between _10_ and _19_, referencing departure times on same day.
When using custom RegEx keep in mind that this is **checking all lines of the dump** (not just the lines of historic_data entries). 

Download latest DB dump and place it in "schema" directory.

In [124]:
! wget -P ../schema/ --show-progress https://dumps.oeffis.0209.cloud/latest.sql.gz

--2023-12-05 13:42:46--  https://dumps.oeffis.0209.cloud/latest.sql.gz
Resolving dumps.oeffis.0209.cloud (dumps.oeffis.0209.cloud)... 116.202.58.77, 2a01:4f8:d1:5a80:209:0:3:106, 2a01:4f8:d1:5a80:209:0:3:109, ...
Connecting to dumps.oeffis.0209.cloud (dumps.oeffis.0209.cloud)|116.202.58.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3657247329 (3.4G) [application/x-gzip]
Saving to: ‘../schema/latest.sql.gz’

latest.sql.gz       100%[===================>]   3.41G  37.2MB/s    in 1m 47s  

2023-12-05 13:44:33 (32.6 MB/s) - ‘../schema/latest.sql.gz’ saved [3657247329/3657247329]



## 1) Extract some set of historic_data entries 
We are limiting the historic data to entries recorded on _2023-11-29_ between hours 10 and 19 (+ entries with departure times on same day only). 
This will extract the DB dump, but keep lines matching recording_time only (will take some time because of full extraction). The result will be just about 300 MB.

In [125]:
! zgrep -e '2023-11-29 1.*t.*2023-11-29' --text ../schema/latest.sql.gz > ../schema/latest_minimized_entries.txt

# 2) Create dump frame (without historic_data entries)
Now you need to insert these historic data entries into the dump file in total (but without the other historic data entries we've filtered before).
Unzip + extract all but historic_data from dump as frame (will take some time because of full extraction). 

In [126]:
! gzip -cd ../schema/latest.sql.gz | sed -n '/^COPY public\.historic_data/,/\\\.$/!p' > ../schema/dump-frame.txt

Fix first line of file (may contain some non-printable chars, instead only use '--').

In [129]:
! { echo "--"; tail -n +2 ../schema/dump-frame.txt; } > temp_file && mv temp_file ../schema/dump-frame.txt

# 3) Insert historic_data entries into frame
Calculate line where historic_data entries should get inserted.

In [130]:
lines = ! grep -n "Data for Name\\: historic_data" ../schema/dump-frame.txt | cut -d: -f1
line_to_insert = int(lines[0]) + 2

Build string that includes starting "COPY" instruction + "\." prefix after entries. Use temp_file to insert into dump-frame afterward.

In [132]:
with open("../schema/latest_minimized_entries.txt", "r") as file:
    historic_data_entries = file.read() 

historic_data_entries = ("COPY public.historic_data (id, trip_id, stop_id, recording_time, is_departure, planned, estimated, vrr_timetable_version_id, trip_code, raw_data, parent_stop_id, is_cancelled) FROM stdin; \n" 
        + historic_data_entries 
        + "\\." + "\n" + "\n")

with open("temp_file", "w") as file:
    file.writelines(historic_data_entries)

In [133]:
! cp ../schema/dump-frame.txt ../schema/latest_minimized.sql
! sed -i "$line_to_insert r temp_file" ../schema/latest_minimized.sql
! rm -f temp_file

# 4) Tidy-up (remove all files but result)

In [135]:
! rm -f ../schema/latest.sql.gz
! rm -f ../schema/latest_minimized_entries.txt
! rm -f ../schema/dump-frame.txt

# 5) Import DB dump
Import created db dump by following instructions in project's README.md.